In [1]:
import pandas as pd

In [3]:
npr=pd.read_csv(r"C:\Users\Prince Shukla\Downloads\npr.csv")

In [5]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


# PREPROCESSING

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
tfidf= TfidfVectorizer(max_df=0.95,min_df=2,stop_words='english')

In [13]:
dtm= tfidf.fit_transform(npr['Article'])

In [15]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

# NMF

In [19]:
from sklearn.decomposition import NMF

In [21]:
nmf_model = NMF(n_components=7,random_state=42)

In [23]:
# this can take a while, we are dealing with a large amount of documents
nmf_model.fit(dtm)

NMF(n_components=7, random_state=42)

# Displaying Topics

In [28]:
len(tfidf.get_feature_names_out())

54777

In [42]:
import random

In [44]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(tfidf.get_feature_names_out()[random_word_id])

sunrise
chapelle
viii
wordplay
doozy
surfer
disappeared
matthews
okla
polymers


In [46]:
len(nmf_model.components_)

7

In [50]:
len(nmf_model.components_[0])

54777

In [52]:
single_topic= nmf_model.components_[0]

In [54]:
# return the indices that would sort the array
single_topic.argsort()

array([    0, 27208, 27206, ..., 36283, 54692, 42993], dtype=int64)

In [58]:
# top 10 words for this topics:
single_topic.argsort()[-10:]

array([14441, 36310, 53989, 52615, 47218, 53152, 19307, 36283, 54692,
       42993], dtype=int64)

In [60]:
top_word_indices = single_topic.argsort()[-10:]

In [64]:
for index in top_word_indices:
    print(tfidf.get_feature_names_out()[index])

disease
percent
women
virus
study
water
food
people
zika
says


In [70]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPICS #{index}')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPICS #0
['new', 'research', 'like', 'patients', 'health', 'disease', 'percent', 'women', 'virus', 'study', 'water', 'food', 'people', 'zika', 'says']


THE TOP 15 WORDS FOR TOPICS #1
['gop', 'pence', 'presidential', 'russia', 'administration', 'election', 'republican', 'obama', 'white', 'house', 'donald', 'campaign', 'said', 'president', 'trump']


THE TOP 15 WORDS FOR TOPICS #2
['senate', 'house', 'people', 'act', 'law', 'tax', 'plan', 'republicans', 'affordable', 'obamacare', 'coverage', 'medicaid', 'insurance', 'care', 'health']


THE TOP 15 WORDS FOR TOPICS #3
['officers', 'syria', 'security', 'department', 'law', 'isis', 'russia', 'government', 'state', 'attack', 'president', 'reports', 'court', 'said', 'police']


THE TOP 15 WORDS FOR TOPICS #4
['primary', 'cruz', 'election', 'democrats', 'percent', 'party', 'delegates', 'vote', 'state', 'democratic', 'hillary', 'campaign', 'voters', 'sanders', 'clinton']


THE TOP 15 WORDS FOR TOPICS #5
['love', 've', 'don

# Attaching Discovered Topic Labels to Original Articles

In [73]:
dtm

<11992x54777 sparse matrix of type '<class 'numpy.float64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [75]:
dtm.shape

(11992, 54777)

In [77]:
len(npr)

11992

In [79]:
topic_results=nmf_model.transform(dtm)

In [81]:
topic_results.shape

(11992, 7)

In [83]:
topic_results[0]

array([0.        , 0.12079653, 0.00139891, 0.05915242, 0.01519226,
       0.        , 0.        ])

In [85]:
topic_results[0].argmax()

1

this means that our model thinks that the first article belongs to topic#1

# Combining with Original Data

In [89]:
npr.head(10)

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."
5,I did not want to join yoga class. I hated tho...
6,With a who has publicly supported the debunk...
7,"I was standing by the airport exit, debating w..."
8,"If movies were trying to be more realistic, pe..."
9,"Eighteen years ago, on New Year’s Eve, David F..."


In [91]:
topic_results.argmax(axis=1)

array([1, 1, 1, ..., 0, 4, 3], dtype=int64)

In [93]:
npr['topic']=topic_results.argmax(axis=1)

In [95]:
npr.head(10)

,Article,topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",3
4,"From photography, illustration and video, to d...",6
5,I did not want to join yoga class. I hated tho...,5
6,With a who has publicly supported the debunk...,0
7,"I was standing by the airport exit, debating w...",0
8,"If movies were trying to be more realistic, pe...",0
9,"Eighteen years ago, on New Year’s Eve, David F...",5
